In [0]:
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.models import load_model

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving inference.zip to inference.zip
User uploaded file "inference.zip" with length 237981778 bytes


In [16]:
!ls

inference  inference.zip  sample_data


In [5]:
!unzip inference.zip

Archive:  inference.zip
  inflating: inference/naive.h5      
  inflating: inference/vgg.h5        
  inflating: inference/x_test.npy    
  inflating: inference/y_test.npy    


In [6]:
!ls

inference  inference.zip  sample_data


In [17]:
os.listdir("inference")

['y_test.npy', 'x_test.npy', 'vgg.h5', 'naive.h5']

In [35]:
x_test = np.load("inference/x_test.npy")
y_test = np.load("inference/y_test.npy")

x_test.shape, y_test.shape

((2756, 200, 200, 3), (2756, 2))

In [0]:
#plt.imshow(x_test[9])

In [20]:
model_1 = load_model("inference/vgg.h5")
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [21]:
model_2 = load_model("inference/naive.h5")
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 128)       73856     
__________

In [22]:
score = model_1.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2756/2756 [==============================] - 1242s 451ms/step
Test loss: 0.13639475404447438
Test accuracy: 0.9499274310595065


In [36]:
score = model_2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2756/2756 [==============================] - 294s 107ms/step
Test loss: 0.25343694605488476
Test accuracy: 0.9499274310595065


In [23]:
# prediction for model 1
y_pred_1 = model_1.predict(x_test, verbose=1)
y_pred_1.shape

2756/2756 [==============================] - 1233s 447ms/step


(2756, 2)

In [25]:
# prediction for model 2
y_pred_2 = model_2.predict(x_test, verbose=1)
y_pred_2.shape

2756/2756 [==============================] - 299s 109ms/step


(2756, 2)

In [37]:
y_pred_1.shape, y_pred_2.shape

((2756, 2), (2756, 2))

In [38]:
# ensemble lol
y_pred = .5 *y_pred_1 + .5 *y_pred_2
y_pred.shape

(2756, 2)

In [39]:
y_pred.shape

(2756, 2)

In [40]:
y_test.shape

(2756, 2)

In [41]:
# get labels from predictions
pred_labels = np.array([np.argmax(pred) for pred in y_pred])
pred_labels.shape

(2756,)

In [42]:
#convert ground truths to column values
y_test = np.argmax(y_test, axis=1)
y_test.shape

(2756,)

In [43]:
y_test.shape, pred_labels.shape

((2756,), (2756,))

In [44]:
print('Accuracy by ensembling two models: ',np.mean((y_test==pred_labels)))

Accuracy by ensembling two models:  0.9582728592162555


In [0]:
# .6 .4 -->>> VGG FTW
# Accuracy by ensembling two models:  0.9589985486211902

# .5 .5
# Accuracy by ensembling two models:  0.9582728592162555